In [4]:
import os
import cv2
import dlib
import numpy as np


In [5]:

input_dir = "/rhinophyma_rendered"
output_dir = "/rhi_masked"
rejected_dir = "/rhi_rejected"


In [6]:
#masking and resizing the images to (224,224) for further to be suitable for classification models

def extract_face(image_path):
    img = cv2.imread(image_path)
    detector = dlib.get_frontal_face_detector()
    faces = detector(img, 1)
    if len(faces) > 0:
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        landmarks = predictor(img, faces[0])
        landmark_tuple = []
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmark_tuple.append((x, y))
        p_1 = (landmark_tuple[36][0], landmark_tuple[60][1]-10)
        p_2 = (landmark_tuple[45][0], landmark_tuple[64][1]-10)
        routes = [p_1, p_2, landmark_tuple[45], landmark_tuple[36]]
        mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        mask = cv2.fillConvexPoly(mask, np.array(routes), 1)
        out = np.zeros_like(img)
        out[mask.astype(bool)] = img[mask.astype(bool)]
        out = cv2.resize(out, (224, 224))
        return out, True
    else:
        return None, False


if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(rejected_dir):
    os.makedirs(rejected_dir)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        rejected_path = os.path.join(rejected_dir, filename)
        face, success = extract_face(input_path)
        if success:
            cv2.imwrite(output_path, face)
        else:
            cv2.imwrite(rejected_path, cv2.imread(input_path))